#### Описание задания

Today you need to predict the house prices in the USA. You can use any model from the course, except Random Forests, Bagging Aggregation, Gradient boosting and any DL models. Use of raw features is not the best idea, but you may create your own ones. Be creative and remember what you were taught!

#### Оценивание

R2 score is used as metric. Public set is only 30% of overall test

The submission file should contain a header and have the following format:

Id,price

1,1000.0

2,9000.0


В соревновании по анализу данных вам предлагается по имеющимся данным решить некоторую задачу, оптимизируя указанную метрику, и отправить ответы для заданного тестового множества. Максимальное количество посылок в сутки ограничено (разрешается сделать 5 посылкок в день), ближе к концу соревнования вам будем необходимо выбрать 1 посылку, которую вы считаете лучшей

В лидербордах каждого из соревнований присутствуют несколько базовых решений (бейзлайнов), каждое из которых соответствует определённой оценке. Например, для получения оценки не ниже 8 баллов необходимо, чтобы ваше решение на приватном лидерборде оказалось лучше соответствующего бейзлайна. Далее для студента, преодолевшего бейзлайн на N1 баллов, но не преодолевшего бейзлайн на N2 балла, итоговая оценка за соревнование рассчитывается по равномерной сетке среди всех таких студентов в зависимости от места в приватном лидерборде среди них; если быть точными, то по следующей формуле:

N2 - (N2 - N_1) * i / M,

где M — количество студентов (из всех студентов, изучающих курс), преодолевших бейзлайн на N1 баллов, но не преодолевших бейзлайн на N2 балла (если студент преодолел максимальный бейзлайн, то N_2 = 10) ;

i — место (начиная с 1) студента в приватном лидерборде среди всех таких студентов.

В течение 3 суток после окончания соревнования в соответствующую форму (вышлем позже) необходимо прислать код, воспроизводящий ответы для посылки, фигурирующей в приватном лидерборде. При оформлении кода предполагайте, что данные лежат рядом с ним в папке data, а в результате выполнения кода ответы должны быть записаны в файл solution-N-Username.csv, где N — номер соревнования, Username — ваша фамилия. У нас должна быть возможность запустить код и получить те же ответы, что и в вашей посылке, — в частности, это означает, что:

    Если вы отправляете файл *.py, мы будем запускать его при помощи команды python *.py в вышеуказанном предположении о местонахождении данных.

    Если вы отправляете ноутбук *.ipynb, мы последовательно запустим все ячейки ноутбука и будем ожидать в результате его работы формирование файла с ответами.


#### Информация о данных


Data fields

    string date – date house was sold
    float32 bedrooms – number of bedrooms
    float32 bathrooms – number of bathrooms/bedrooms
    int32 sqft_living – square footage of the home
    int32 sqft_lot – square footage of the lot
    float32 floors – total floors (levels) in house
    bool waterfront – house which has a view to a waterfront
    int32 view - Has been viewed
    int32 condition – how good the condition is (overall)
    int32 grade – overall grade given to the housing unit
    int32 sqft_above – square footage of house apart from basement
    int32 sqft_basement – square footage of the basement
    int32 yr_built – built Year
    int32 yt_renovated – year when house was renovated
    int32 zipcode
    float32 lat – latitude coordinate
    float32 long – longitude coordinate


In [ ]:
N = 4
folder = './data/'

In [ ]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import StackingRegressor

import matplotlib.pyplot as plt
import seaborn as sns

#### Импортируем исходные данные

In [ ]:
X_train = pd.read_csv(f'{folder}x_train.csv')
X_test = pd.read_csv(f'{folder}x_test.csv')
y_train = pd.read_csv(f'{folder}y_train.csv')

#### Исследуем данные: Целевая переменная

In [ ]:
sns.histplot(y_train);

Преобразуем в логарифм. Далее будем предсказывать логарифм цены, и затем потенцируем предсказание

In [ ]:
y_train_log = np.log(y_train)

In [ ]:
sns.histplot(y_train_log);

Получили распределение, более похожее на нормальное

#### Соберем все данные в один датасет

In [ ]:
# убираем явный выброс по критерию bedrooms == 33, при этом площадь всего 1620 sqft

X_train.query('bedrooms > 30')

In [ ]:
X_train['dataset'] = 'train'
X_test['dataset'] = 'test'

data = pd.concat([pd.concat([X_train.drop(2113), y_train_log.drop(2113)], axis=1), X_test])

In [ ]:
data.info()

In [ ]:
data.head()

Проверим, насколько совпадают распределния для тренировочных и тестовых данных

In [ ]:
for col in data.columns[:-2]:
    sns.histplot(X_train[col], color='red')
    sns.histplot(X_test[col], color='green')
    plt.show()

Распределения похожи

##### Признаки

In [ ]:
X_train.info()

In [ ]:
X_test.info()

Пропусков в данных нет

In [ ]:
X_train.head()

In [ ]:
X_train.describe().drop('count').style.bar()

In [ ]:
# так как мало значений для view, sqft_basement, yr_renovated, делаем их булевыми

for col in ['view', 'sqft_basement', 'yr_renovated']:
    data[col + '_bool'] = (data[col] > 0).astype(int)
    data.drop(col, axis=1, inplace=True)

In [ ]:
# переводим "квадратные" признаки в сторону квадрата (извлекаем корень) и логарифмируем

for col in data.columns:
    if col.startswith('sqft') and not col.endswith('bool'):
        data[col[2:]] = np.log(np.sqrt(data[col]))
        data.drop(col, axis=1, inplace=True)

In [ ]:
# извлекаем дату продажи и переводим ее в дни, начиная с самого раннего дня продажи

data.date = pd.to_datetime(data.date.str.split('T', expand=True)[0], format='%Y%m%d')

min_data = data.date.min()
data.date = (data.date - min_data).apply(lambda x: x.days)

In [ ]:
data.columns

In [ ]:
#(X_train.sqft_above + X_train.sqft_basement - X_train.sqft_living).describe()

### Категориальная переменная zipcode

Собираем общую информацию по каждому zipcode

In [ ]:
from tqdm import tqdm

from selenium import webdriver
from selenium.webdriver.common.by import By
import time

In [ ]:
d_zipcodes_info = {}
info_fields = ['Population', 'Population Density', 'Housing Units', 'Median Home Value', 'Land Area', 'Water Area',
              'Occupied Housing Units', 'Median Household Income', 'Median Age',
              ]

SCROLL_PAUSE_TIME = 1
driver = webdriver.Chrome()

for zipcode in tqdm(data.zipcode.unique()):
    url = f'https://www.unitedstateszipcodes.org/{zipcode}/'
    driver.get(url)

    time.sleep(SCROLL_PAUSE_TIME)
    list_p_element = driver.find_elements(By.XPATH, "//tr")

    d_zipcodes_info[zipcode] = {}
    fulltxt = []
    
    for i_start, el in enumerate(list_p_element):
        txt = el.text
        if txt.startswith('Population'):
            break

    i = -1
    for el in list_p_element:
        i += 1
        if i < i_start or i_start + 8 < i:
            continue
        txt = el.text
        if txt:
            fulltxt += [txt]

    for txt, field in zip(fulltxt, info_fields):
        value = float(txt.split(field)[1].replace(':', '').strip().split()[0].replace(',', '').replace('$', ''))
        d_zipcodes_info[zipcode][field] = value
        
for field in info_fields:
    colname = '_'.join(field.split())
    data[colname] = data.zipcode.apply(lambda x: d_zipcodes_info[x][field])

In [ ]:
# write dict into file

import json

d_zipcodes_modified_keys = {str(key):val for key, val in d_zipcodes_info.items()}

with open('d_zipcodes_info.json', 'w') as fd:
    json.dump(d_zipcodes_modified_keys, fd)

In [ ]:
# load dict from file

import json

with open('d_zipcodes_info.json', 'к') as fd:
    json.load(d_zipcodes_modified_keys, fd)
    
d_zipcodes_info = {int(key):val for key, val in d_zipcodes_info.items()}

In [ ]:
for col in data.columns:
    sns.histplot(data[col], color='green')
    plt.show()

In [ ]:
for col in data.columns:
    sns.scatterplot(data.query('dataset == "train"')[col], data.query('dataset == "train"')['price'], color='green')
    plt.show()

In [ ]:
# немного преобразований переменных

data['Population'] = np.sqrt(data['Population'])
data['pers_houses_occupied'] = data.Occupied_Housing_Units / data.Housing_Units

#### проверим есть ли одинаковые дома

In [ ]:
variables=['bedrooms', 'bathrooms', 'floors', 'waterfront', 'condition', 'grade', 'yr_built', 'zipcode', 
           'lat', 'long', 'view_bool', 'sqft_basement_bool', 'yr_renovated_bool', 'ft_living', 'ft_lot', 'ft_above'
]

data['combined'] = data[variables].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

In [ ]:
vals, counts = np.unique(data['combined'], return_counts=True)

In [ ]:
counts.sort()

In [ ]:
len(counts[counts > 1])

Есть один и тот же дом несколько раз

In [ ]:
data.to_csv('data_all.csv', index=False)

#### Делим данные на тест и трейн

Переделать

In [ ]:
X_train = data.query('~price.isna()').drop([
    #'lat', 'long', 'zipcode',
    'dataset', 'price', 'combined'], axis=1)
X_test = data.query('price.isna()').drop([
    #'lat', 'long', 'zipcode',
    'dataset', 'price', 'combined'], axis=1)
y_train = data.query('~price.isna()')[['price']]

#### Подготовка категориальных данных

In [ ]:
cat_features = ['zipcode', 
                #'view_bool', 'sqft_basement_bool', 'waterfront', 'yr_renovated_bool'
               ]

In [ ]:
from sklearn import preprocessing

for col in cat_features:
    le = preprocessing.LabelEncoder()
    le.fit(X_train[col])
    X_train[col] = le.transform(X_train[col])
    X_test[col] = le.transform(X_test[col])

#le.inverse_transform([0, 0, 1, 2])

#### Удаляем коррелирующие признаки

In [ ]:
correlated_features = pd.DataFrame(X_train.corr(method='pearson'
                                         ).abs().unstack().reset_index().query('level_0 != level_1'
                                                                              ).sort_values(0, ascending=False))

In [ ]:
correlated_features['pairs'] = correlated_features.apply(lambda x: tuple(sorted([x.level_0, x.level_1])), axis=1)

In [ ]:
correlated_features = correlated_features.drop_duplicates(subset='pairs').drop(['level_0', 
                                                          'level_1'], axis=1).rename(columns={0:'corr_value'})

In [ ]:
correlated_features.corr_value.hist(bins=100)

In [ ]:
from collections import Counter
import itertools

In [ ]:
# уберем из тех пар, у которых корреляция > corr те признаки, которые чаще других встречаются в парах 
# коррелирующих признаков

corr = 0.8
#corr = 0.7
f_to_drop = []
most_correlated_df = correlated_features.query('corr_value > @corr').copy()

# пока есть пары в датасете с самыми коррелированными фичами
while len(most_correlated_df.index):
    # список всех оставшихся коррелированных фичей в датасете
    most_corr = list(itertools.chain(*most_correlated_df.pairs))
    # самая часто встречающаяся фича
    f = np.unique(sorted(most_corr, key=lambda x: Counter(most_corr)[x]))[-1]
    f_to_drop += [f]
    most_correlated_df['f_in_pairs'] = most_correlated_df.pairs.apply(lambda x: f in x)
    most_correlated_df = most_correlated_df.query('not f_in_pairs')

In [ ]:
correlated_features.query('corr_value > @corr').copy()

In [ ]:
len(f_to_drop)

In [ ]:
f_to_drop

In [ ]:
X_train.drop(f_to_drop, axis=1, inplace=True)
X_test.drop(f_to_drop, axis=1, inplace=True)

#### Делим датасет на трейн и валидацию

In [ ]:
X_train_train, X_val, y_train_train, y_val = train_test_split(X_train, y_train, random_state=42)

#### Шкалируем переменные с помощью Standard Scaler

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train_train_scaled = pd.DataFrame(scaler.transform(X_train_train), columns=X_train_train.columns)
X_val_scaled = pd.DataFrame(scaler.transform(X_val), columns=X_val.columns)
X_train_scaled = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

In [ ]:
X_train_scaled.describe()

#### Поиск лучших параметров

In [ ]:
# поиск лучших параметров для регрессии

grid_search_lasso = GridSearchCV(Lasso(), {'max_iter': range(10, 150, 10), 'alpha': np.logspace(-9, -5), 'random_state':[42]}, scoring='r2')
grid_search_lasso.fit(X_train_scaled, y_train)
grid_search_lasso.best_params_

In [ ]:
# поиск лучших параметров для knn

grid_search_knn = GridSearchCV(KNeighborsRegressor(), 
                           {'metric': ['cosine', 'euclidean', 'manhattan', 'chebyshev', 
                                       'hamming', 'canberra', 'braycurtis'], 
                            'weights': ['distance'], 
                            'n_neighbors': range(3, 8)}, scoring='r2')
grid_search_knn.fit(X_train_scaled, y_train)
grid_search_knn.best_params_

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
 
# get a stacking ensemble of models
def get_stacking():
    # define the base models
    level0 = list()
    #level0.append(('knn', KNeighborsRegressor(weights='distance', metric='braycurtis', n_neighbors=7)))
    level0.append(('knn', KNeighborsRegressor(**grid_search_knn.best_params_)))
    #level0.append(('lr', LinearRegression()))
    #level0.append(('lasso', Lasso(alpha=0.001, random_state=42)))
    level0.append(('lasso', Lasso(**grid_search_lasso.best_params_)))
    #level0.append(('ridge', Ridge()))
    # define meta learner model
    level1 = LinearRegression()
    # define the stacking ensemble
    model = StackingRegressor(estimators=level0, final_estimator=level1, cv=5)
    return model
 
# # get a list of models to evaluate
# def get_models():
#     models = dict()
#     models['knn'] = KNeighborsRegressor(**grid_search_knn.best_params_)
#     #models['lr'] = LinearRegression()
#     models['lasso'] = Lasso(**grid_search_lasso.best_params_)
#     #models['ridge'] = Ridge()
#     models['stacking'] = get_stacking()
#     return models
 
# # evaluate a given model using cross-validation
# def evaluate_model(model, X, y):
#     cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)
#     scores = cross_val_score(model, X, y, scoring='r2', cv=cv, n_jobs=-1, error_score='raise')
#     return scores

# get the models to evaluate
# models = get_models()
# # evaluate the models and store results
# results, names = list(), list()
# for name, model in models.items():
#     scores = evaluate_model(model, X_train_scaled, y_train)
#     results.append(scores)
#     names.append(name)
#     print('>%s %.3f (%.3f)' % (name, np.mean(scores), np.std(scores)))
# # plot model performance for comparison
# plt.boxplot(results, labels=names, showmeans=True)
# plt.show()

In [ ]:
model = get_stacking()
# fit the model on all available data
model.fit(X_train_scaled, y_train)
# make a prediction for one example
y_pred = model.predict(X_test_scaled)

In [ ]:
submission = pd.DataFrame(np.exp(y_pred), columns=['price']).reset_index()

In [ ]:
submission.columns = ['Id', 'price']

In [ ]:
submission.set_index('Id').to_csv(f'solution-{N}-Uliana_Bykova.csv')

In [ ]:
# 0.858 - 0.857 - 0.879